In [1]:
import string
import glob
import json
import numpy as np
import pandas as pd
import itertools

In [2]:
def preprocess_entity(entity):
    return entity.lower().translate(str.maketrans('', '', string.punctuation + ' '))

def align_prediction(predictions, reference_ids):
    result = {ref_id: predictions[ref_id] for ref_id in reference_ids}
    return result

def remove_first_api_call(predictions):
    result_dict = {}
    for key, prediction in predictions.items():
        new_prediction = []
        is_skip = False
        for turn in prediction:
            if 'api_call' in turn['text'] and not is_skip:
                is_skip = True
            else:
                new_prediction.append(turn)
        result_dict[key] = new_prediction
    return result_dict
            
def extract_goal_from_gold(golds, references, reference_ids):
    # Build dialogue id to key string dict from references
    id_to_key_dict = {}
    for ref_id in reference_ids:
        dialogue = references[int(ref_id)]
        usr_dial = []
        for turn in dialogue:
            if turn['spk'] == 'USR':
                usr_dial.append(turn['text'].translate(str.maketrans('', '', string.punctuation + ' ')))
        id_to_key_dict[ref_id] = ''.join(usr_dial)
        
    # Build key string to constraint+goal dict from golds
    key_to_goal_dict = {}
    key_to_dial_dict = {}
    for dialogue in golds:
        usr_dial = []
        for turn in dialogue['dial']:
            usr_dial.append(turn['usr']['transcript'].lower().translate(str.maketrans('', '', string.punctuation + ' ')))
        key_to_goal_dict[''.join(usr_dial)] = {'constraint': dialogue['goal']['constraints'], 'request':dialogue['goal']['request-slots']}
        key_to_dial_dict[''.join(usr_dial)] = dialogue
        
    # Matching key between two dicts
    id_to_goal_dict = {}
    id_to_dial_dict = {}
    for dial_id, str_key in id_to_key_dict.items():
        id_to_goal_dict[dial_id] = key_to_goal_dict[str_key]
        id_to_dial_dict[dial_id] = key_to_dial_dict[str_key]
    
    return id_to_goal_dict, id_to_dial_dict

In [3]:
# Load KBret
kbret_json = json.load(open('../runs/CAMREST_flattenKB_False_kbpercentage_KBret/result.json','r'))
kbret_ids = list(kbret_json.keys())

In [4]:
# Load model prediction
model_prediction_dict = {'KBret': kbret_json}
for f in glob.glob('../runs/CAMREST_gpt2_graph_False_adj_False_edge_False_unilm_False_*/result.json'):
    json_data = json.load(open(f))
    
    # Get Name
    flattenKB = eval(f[f.index('flattenKB_') + len('flattenKB_'):].split('_')[0])
    if flattenKB:
        model_name = 'GPT2+KB'
    else:
        if 'kbpercentage_' in f:
            kbpercentage = f[f.index('kbpercentage_') + len('kbpercentage_'):].split('_')[0]
            model_name = f'GPT2+KA{kbpercentage}'
        else:
            raise ValueError('NO NO NO NO!!!!')
        
    # Preprocess
    result_json = align_prediction(json_data, kbret_ids)
    if model_name == 'GPT2+KB':
        result_json = remove_first_api_call(result_json)
        
    # Push to buffer
    model_prediction_dict[model_name] = result_json

In [5]:
# Load human prediction
test_file = open('../data/CamRest/test.txt','r')

human_json = {}
sys_utterances = []
for line in test_file:
    if line == "\n":
        # Push to buffer
        human_json[str(len(human_json))] = sys_utterances
        
        # Reinit variable
        sys_utterances = []
    else:
        _, line = line.replace("\n","").split(' ', 1)
        if "\t" in line:            
            _, syst = line.split("\t")
            if 'api_call' not in syst:
                sys_utterances.append({'spk':'SYS','text':syst})       
if len(sys_utterances) > 0:
    # Push to buffer
    human_json[str(len(human_json))] = sys_utterances

# Preprocess
human_json = align_prediction(human_json, kbret_ids)

# Push to model buffer
model_prediction_dict['Human'] = human_json

In [6]:
# Extract user goal
gold_json = json.load(open('./CamRest676.json','r'))
gen_kbret_json  = json.load(open('../runs/CAMREST_flattenKB_False_kbpercentage_KBret//gen_dialogue.json'))
id_to_goal_dict, gold_dial =  extract_goal_from_gold(gold_json, gen_kbret_json, kbret_ids)

In [7]:
# Measure success rate
KB = pd.DataFrame.from_records(json.load(open('../data/CamRest/KB.json','r')))
KB = KB.fillna('0123456789876543210')
KB = KB.applymap(preprocess_entity)

data = []
success_rate_data = []
for model_name, predictions in model_prediction_dict.items():
    for dial_id, prediction in predictions.items():
        goal = id_to_goal_dict[dial_id]
        
        # Find all valid entities in KB
        subset_KB = KB
        constraints, request = goal['constraint'], goal['request']
        for constraint in constraints:
            constraint_value = preprocess_entity(constraint[1])
            if constraint_value != 'dontcare':
                subset_KB = subset_KB.loc[subset_KB[constraint[0]].str.contains(constraint_value),:]
        entity_list = subset_KB.loc[:,request].values.tolist()
        
        # Find all terms
        terms = []
        for turn in prediction:
            terms += list(map(preprocess_entity, turn['text'].split()))
        terms = ''.join(terms)
        
        # Check success goal
        success = 0
        for i, entity_attributes in enumerate(entity_list):
            num_attr = len(entity_attributes)
            found_attr = 0
            for j, entity_attribute in enumerate(entity_attributes):
                if entity_attribute in terms:
                    found_attr += 1
                    
            if found_attr == num_attr:
                success = 1
                break
    
        if success == 0 and model_name == 'Human':
            data.append({'dial_id':dial_id, 'ent': entity_list, 'constraint':constraints, 'request':request, 'KB':subset_KB.copy(), 'success':success})
            
        # Push success goal data
        success_rate_data.append({'model_name':model_name, 'dial_id':dial_id, 'success':success})

In [8]:
df = pd.DataFrame.from_records(success_rate_data)
success_df = df.groupby('model_name')['success'].sum() / df['dial_id'].nunique() * 100
success_df.to_csv('camrest_success_rate.csv', index=False)

In [9]:
success_df

model_name
GPT2+KA0      30.379747
GPT2+KA10     62.025316
GPT2+KA100    72.151899
GPT2+KA161    74.683544
GPT2+KA50     70.886076
GPT2+KB       62.025316
Human         86.075949
KBret         62.025316
Name: success, dtype: float64